# Visualização Interativa de Mudanças no NDVI ao Longo dos Anos

Este notebook carrega várias imagens NDVI de diferentes anos, gera previsões usando ARIMA e exibe as mudanças ao longo do tempo em um mapa interativo. Através do controle de camadas, você poderá alternar entre diferentes anos e visualizar os dados.

## Passos:
1. Carregar dados NDVI das imagens .tif de diferentes anos.
2. Exibir os NDVIs de diferentes anos em um mapa interativo com controle de camadas.
3. Permitir comparação entre os anos por meio da alternância de camadas.


In [15]:

import os
import numpy as np
import rasterio
import folium
import matplotlib.pyplot as plt


In [16]:

def create_ndvi_layer(image_file, map_center, year, folium_map, opacity=0.6):
    with rasterio.open(image_file) as src:
        ndvi_image = src.read(1)
        ndvi_bounds = src.bounds

        # Salvar a imagem temporariamente como PNG para exibir no mapa
        plt.figure(figsize=(8, 6))
        img = plt.imshow(ndvi_image, cmap='RdYlGn', vmin=np.nanmin(ndvi_image), vmax=np.nanmax(ndvi_image))
        plt.colorbar(img, label='NDVI')
        plt.title(f'NDVI - {year}')
        output_image = f'./resultados/ndvi_{year}.png'
        plt.savefig(output_image)
        plt.close()

        # Adicionar a imagem como uma camada ao mapa
        ndvi_overlay = folium.raster_layers.ImageOverlay(
            image=output_image,
            bounds=[[ndvi_bounds.bottom, ndvi_bounds.left],
                    [ndvi_bounds.top, ndvi_bounds.right]],
            opacity=opacity,
            name=f'NDVI {year}'
        )
        ndvi_overlay.add_to(folium_map)


In [17]:

def create_interactive_ndvi_map(image_folder, map_center):
    # Criar o mapa base
    folium_map = folium.Map(location=map_center, zoom_start=10)

    # Obter todos os arquivos NDVI na pasta (assumindo que o nome do arquivo inclui o ano)
    ndvi_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.tif')])

    # Iterar sobre os arquivos NDVI, adicionando cada um como uma camada separada no mapa
    for ndvi_file in ndvi_files:
        year = ndvi_file.split('.')[0]  # Assumindo que o ano está no nome do arquivo (ex: "NDVI_2020.tif")
        file_path = os.path.join(image_folder, ndvi_file)
        create_ndvi_layer(file_path, map_center, year, folium_map)

    # Adicionar controle de camadas para alternar entre os anos
    folium.LayerControl().add_to(folium_map)

    # Retornar o mapa interativo
    return folium_map


In [18]:

def main(image_folder, output_folder, map_center):
    # Criar o mapa interativo com os NDVIs de anos diferentes
    ndvi_map = create_interactive_ndvi_map(image_folder, map_center)

    # Salvar o mapa interativo como arquivo HTML
    output_file = os.path.join(output_folder, 'ndvi_comparison_map.html')
    ndvi_map.save(output_file)
    print(f"Mapa interativo salvo em {output_file}")


In [19]:

# Exemplo de uso
image_folder = "./Angola_MODIS_Images"  # Pasta com as imagens NDVI (por ano)
output_folder = "./resultados"  # Pasta para salvar resultados
map_center = [-12.5, 17.5]  # Coordenadas do centro do mapa (latitude, longitude)

main(image_folder, output_folder, map_center)


Mapa interativo salvo em ./resultados/ndvi_comparison_map.html
